In [11]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [12]:
house_dict = []

# Filter Out Coming Soon, Auction, Home Type: Apartment, Days On Zillow: 90 Days 
# linked = 'https://www.zillow.com/ca/duplex/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22ca%22%2C%22mapBounds%22%3A%7B%22west%22%3A-129.21631596875%2C%22east%22%3A-109.39698003125%2C%22south%22%3A30.747427935367085%2C%22north%22%3A43.5453012351376%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A9%2C%22regionType%22%3A2%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22sf%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22doz%22%3A%7B%22value%22%3A%2290%22%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A6%7D'
linked = 'https://www.zillow.com/ca/duplex/?searchQueryState=%7B%22usersSearchTerm%22%3A%22ca%22%2C%22mapBounds%22%3A%7B%22west%22%3A-129.21631596875%2C%22east%22%3A-109.39698003125%2C%22south%22%3A30.747427935367085%2C%22north%22%3A43.5453012351376%7D%2C%22mapZoom%22%3A6%2C%22category%22%3A%22cat2%22%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A9%2C%22regionType%22%3A2%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22doz%22%3A%7B%22value%22%3A%2290%22%7D%2C%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22sf%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%7D'
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKait/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'
headers = {'User-Agent': user_agent}
response = requests.get(linked, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
results = soup.find_all('article', class_='list-card')
    
# Loop through returned results
for result in results:
#     print(result)
    # Error handling
    try:
        # Identify and return housing weblink
        house_website = str(result.a.get('href'))
        print(house_website)
        # Identify and return house data
#         listing_type = result.find('div', class_="list-card-footer").text;
        price = result.find('div', class_="list-card-price").text
#         print(price);
        full_address = result.find('address', class_="list-card-addr").text
        street = result.find('address', class_="list-card-addr").text.split(",")[0]
        city = result.find('address', class_="list-card-addr").text.split(",")[1].replace(" ", "")
        state = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[1]
        try:
            zip_code = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[2]
        except IndexError:
            zip_code = 'null'
        bed = result.find('ul', class_="list-card-details").find('li').text.split(" ")[0]
        bath = result.find('ul', class_="list-card-details").select('li')[1].text.split(" ")[0]
        house_square_feet = result.find('ul', class_="list-card-details").select('li')[2].text.split(" ")[0]
        # Loop inside each house website and get more house data
        loop_response = requests.get(house_website, headers=headers)
        loop_soup = BeautifulSoup(loop_response.text, 'html.parser')
        
        loop_result1 = loop_soup.find('ul', class_="ds-home-fact-list")
        loop_result2 = loop_soup.findAll('li', class_="ds-home-fact-list-item")
        
        lot = 'null';
        house_type = 'null';
        year_build = 'null';
        parking = 'null';
        hoa = 'null';
        lot_conversion_to_sqft = 'null';
        zillow_price_per_sqft = 'null';

        lot_search = loop_soup.findAll('div', class_="sc-pJipy hgzjTs")
        for content in lot_search:
#             print(content.text)
            if "LotLot size" in content.text.split(":")[0]:
                lot = content.text.split(": ")[1];
                lot = lot.replace('Lot features', "")
                lot = lot.replace('Lot size dimensions', "")
                    
        for content in loop_result2:
            if "Lot" in content.text.split(":")[0]:
                lot = content.text.split(":")[1];
            if "Type" in content.text.split(":")[0]:
                house_type = content.text.split(":")[1];
            if "Year built" in content.text.split(":")[0]:
                year_build = content.text.split(":")[1];
            if "Parking" in content.text.split(":")[0]:
                parking = content.text.split(":")[1];
            if "HOA" in content.text.split(":")[0]:
                hoa = content.text.split(":")[1];
            if "Price/sqft" in content.text.split(":")[0]:
                zillow_price_per_sqft = content.text.split(":")[1];
        try:
            if (str(lot.split(" ")[1]) == "sqft"):
                lot_conversion_to_sqft = float(lot.split(" ")[0].replace(',', ""));
            elif ((str(lot.split(" ")[1]) == "acres") or (str(lot.split(" ")[1]) == "acre")) or (str(lot.split(" ")[1]) == "Acres"):
                lot_conversion_to_sqft = round(float(lot.split(" ")[0].replace(',', "")) * 43560, 2);
            elif (lot == "No Data"):
                lot_conversion_to_sqft = 'null'
            else:
                lot_conversion_to_sqft = 'null'
        except IndexError:
            lot_conversion_to_sqft = 'null'
#         print(loop_soup.findAll('span', class_="Text-c11n-8-18-0__aiai24-0 einFCw"))
        if (len(loop_soup.findAll('span', class_="Text-c11n-8-38-0__aiai24-0 cMFTfb")) < 3):
            zestimate = 'null'
            rent_zestimate = 'null'
        else:
            zestimate = loop_soup.findAll('span', class_="Text-c11n-8-38-0__aiai24-0 cMFTfb")[0].text
            rent_zestimate = loop_soup.findAll('span', class_="Text-c11n-8-38-0__aiai24-0 cMFTfb")[2].text
        if (zestimate[0] != '$'):
            zestimate = 'null';
        print(zestimate, 'test')
#                 print('zestimate', zestimate)
        try:
            rent_zestimate = loop_soup.findAll('span', class_="Text-c11n-8-33-0__aiai24-0 LoSpP")[1].text
        except:
            rent_zestimate = 'null'
        print(rent_zestimate, 'test')

#                 print('zestimate', zestimate)
#             rent_zestimate = 'null'
#             print(zestimate)

            # Store data to house object if all elements are available
#             if (house_website and listing_type and price and full_address and street and city and state and zip_code and bed and bath and house_square_feet and house_type and year_build):
        house_obj = {
#             "Listing_type": listing_type,
            "Price": price,
            "Full_address": full_address,
            "Street": street,
            "City": city,
            "State": state,
            "Zip_code": zip_code,
            "Bed": bed,
            "Bath": bath,
            "Square_Feet": house_square_feet,
            "Lot": lot,
            "Lot_conversion_to_sqft": lot_conversion_to_sqft,
            "House_type": "Apartment",
            "Year_build": year_build,
            "Parking": parking,
            "HOA": hoa,
            "Zillow_price/sqft": zillow_price_per_sqft,
            "Zestimate": zestimate,
            "Rent_zestimate": rent_zestimate,
            "House_website": house_website
            }

        #Continue to add data into collection house_dict
        house_dict.append(house_obj)
#         print(house_dict)
        time.sleep(5)
    except AttributeError as e:
        continue

# print(house_dict)

https://www.zillow.com/homedetails/5954-Rancho-Mission-Rd-UNIT-189-San-Diego-CA-92108/16939396_zpid/
$312,900 test
null test
https://www.zillow.com/homedetails/151-Buckingham-Dr-UNIT-267-Santa-Clara-CA-95051/19600998_zpid/
null test
null test
https://www.zillow.com/homedetails/1000-Evergreen-Ter-San-Pablo-CA-94806/2124672524_zpid/
null test
null test
https://www.zillow.com/homedetails/6600-Warner-Ave-UNIT-166-Huntington-Beach-CA-92647/25734862_zpid/
$403,500 test
null test
https://www.zillow.com/homedetails/1979-Clay-St-San-Francisco-CA-94109/61288306_zpid/
$2,150,400 test
null test
https://www.zillow.com/homedetails/39821-Cedar-Blvd-UNIT-302-Newark-CA-94560/25061442_zpid/
null test
null test
https://www.zillow.com/homedetails/321-10th-Ave-UNIT-1007-San-Diego-CA-92101/2081178210_zpid/
$775,400 test
null test
https://www.zillow.com/homedetails/3614-4th-Ave-San-Diego-CA-92103/2096832417_zpid/
null test
null test
https://www.zillow.com/homedetails/150-Via-Robles-Paso-Robles-CA-93446/20808

In [13]:
house_data = pd.DataFrame(house_dict)
house_data

,Price,Full_address,Street,City,State,Zip_code,Bed,Bath,Square_Feet,Lot,Lot_conversion_to_sqft,House_type,Year_build,Parking,HOA,Zillow_price/sqft,Zestimate,Rent_zestimate,House_website
0,"$330,000","5954 Rancho Mission Rd UNIT 189, San Diego, CA...",5954 Rancho Mission Rd UNIT 189,SanDiego,CA,92108,1,1,530,7.56 Acres,329314,Apartment,1977,2 Parking spaces,$275 monthly,$623,"$312,900",null,https://www.zillow.com/homedetails/5954-Rancho...
1,"$550,000","151 Buckingham Dr UNIT 267, Santa Clara, CA 95051",151 Buckingham Dr UNIT 267,SantaClara,CA,95051,1,1,770,770 sqft,770,Apartment,1969,2 Parking spaces,$443 monthly,$714,null,null,https://www.zillow.com/homedetails/151-Bucking...
2,"$320,000","1000 Evergreen Ter, San Pablo, CA 94806",1000 Evergreen Ter,SanPablo,CA,94806,2,2,"1,136",null,null,Apartment,1984,1 Parking space,$450 monthly,$282,null,null,https://www.zillow.com/homedetails/1000-Evergr...
3,"$385,000","6600 Warner Ave UNIT 166, Huntington Beach, CA...",6600 Warner Ave UNIT 166,HuntingtonBeach,CA,92647,1,1,785,null,null,Apartment,1969,1 Parking space,$420 monthly,$490,"$403,500",null,https://www.zillow.com/homedetails/6600-Warner...
4,"$2,700,000","1979 Clay St, San Francisco, CA 94109",1979 Clay St,SanFrancisco,CA,94109,4,3,"2,546",null,null,Apartment,1938,1 Parking space,$600 monthly,"$1,060","$2,150,400",null,https://www.zillow.com/homedetails/1979-Clay-S...
5,"$599,000","39821 Cedar Blvd UNIT 302, Newark, CA 94560",39821 Cedar Blvd UNIT 302,Newark,CA,94560,2,2,"1,071",6.32 Acres,275299,Apartment,1986,2 Parking spaces,$420 monthly,$559,null,null,https://www.zillow.com/homedetails/39821-Cedar...
6,"$780,000","321 10th Ave UNIT 1007, San Diego, CA 92101",321 10th Ave UNIT 1007,SanDiego,CA,92101,2,2,"1,103",null,null,Apartment,2007,2 Parking spaces,$677 monthly,$707,"$775,400",null,https://www.zillow.com/homedetails/321-10th-Av...
7,"$275,000","3614 4th Ave, San Diego, CA 92103",3614 4th Ave,SanDiego,CA,92103,2,1,"1,300",2 sqft,2,Apartment,1930,"Garage - Detached, Off-street","$1,000 monthly",$212,null,null,https://www.zillow.com/homedetails/3614-4th-Av...
8,"$188,000","150 Via Robles, Paso Robles, CA 93446",150 Via Robles,PasoRobles,CA,93446,2,2,800,null,null,Apartment,1979,Carport,$633 monthly,$235,"$521,900",null,https://www.zillow.com/homedetails/150-Via-Rob...


In [14]:
# Export file as a CSV, without the Pandas index, but with the header
# house_data.to_csv("Output/Zillow/Q2-2021_Apartment_AgentListing_page1.csv", index=False, header=True)
house_data.to_csv("Output/Zillow/Q2-2021_Apartment_OtherListing_page1.csv", index=False, header=True)